In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np

In [2]:
# 1. 准备数据
np.random.seed(42)
n_samples = 1000  # 增加样本数量以更好地展示批处理效果
x_np = np.random.randn(n_samples, 1)
y_np = 2 * x_np + 3 + 0.1 * np.random.randn(n_samples, 1)

# 转换为PyTorch张量
x = torch.from_numpy(x_np).float()
y = torch.from_numpy(y_np).float()

In [3]:
# 2. 划分训练集和测试集
train_ratio = 0.8
train_size = int(train_ratio * n_samples)
test_size = n_samples - train_size

x_train, x_test = x[:train_size], x[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

print(f"训练集大小: {train_size}, 测试集大小: {test_size}")

训练集大小: 800, 测试集大小: 200


In [4]:
# 3. 创建数据集和数据加载器
batch_size = 32

# 创建TensorDataset
train_dataset = TensorDataset(x_train, y_train)
test_dataset = TensorDataset(x_test, y_test)

# 创建DataLoader
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=0)

In [5]:
# 4. 定义模型
class LinearRegression(nn.Module):
    def __init__(self):
        super(LinearRegression, self).__init__()
        self.linear = nn.Linear(1, 1)  # 输入维度1，输出维度1
        
    def forward(self, x):
        return self.linear(x)


In [6]:
# 5. 创建模型实例、定义损失函数和优化器
model = LinearRegression()
# 打印初始参数
print(f"初始参数: w = {model.linear.weight.item():.3f}, b = {model.linear.bias.item():.3f}")

criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

初始参数: w = 0.445, b = 0.500


In [7]:
# 6. 训练循环（使用小批量加载）
n_epochs = 100
losses = []
val_losses = []

for epoch in range(n_epochs):
    # 训练模式
    model.train()
    epoch_loss = 0.0
    
    # 批量训练
    for batch_idx, (batch_x, batch_y) in enumerate(train_loader):
        # 前向传播
        batch_pred = model(batch_x)
        
        # 计算损失
        loss = criterion(batch_pred, batch_y)
        
        # 反向传播和优化
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
    
    # 计算平均训练损失
    avg_train_loss = epoch_loss / len(train_loader)
    losses.append(avg_train_loss)
    
    # 评估模式
    model.eval()
    val_loss = 0.0
    
    with torch.no_grad():
        for batch_x, batch_y in test_loader:
            batch_pred = model(batch_x)
            val_loss += criterion(batch_pred, batch_y).item()
    
    # 计算平均验证损失
    avg_val_loss = val_loss / len(test_loader)
    val_losses.append(avg_val_loss)
    
    # 每10轮打印一次损失
    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{n_epochs}], Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}')



Epoch [10/100], Train Loss: 0.0107, Val Loss: 0.0097
Epoch [20/100], Train Loss: 0.0100, Val Loss: 0.0093
Epoch [30/100], Train Loss: 0.0100, Val Loss: 0.0093
Epoch [40/100], Train Loss: 0.0100, Val Loss: 0.0093
Epoch [50/100], Train Loss: 0.0100, Val Loss: 0.0093
Epoch [60/100], Train Loss: 0.0100, Val Loss: 0.0093
Epoch [70/100], Train Loss: 0.0100, Val Loss: 0.0093
Epoch [80/100], Train Loss: 0.0100, Val Loss: 0.0093
Epoch [90/100], Train Loss: 0.0100, Val Loss: 0.0093
Epoch [100/100], Train Loss: 0.0100, Val Loss: 0.0093


In [8]:
# 7. 输出最终参数
print(f"\n训练完成!")
print(f"最终参数: w = {model.linear.weight.item():.4f}, b = {model.linear.bias.item():.4f}")
print(f"真实参数: w = 2.0, b = 3.0")


训练完成!
最终参数: w = 1.9943, b = 3.0081
真实参数: w = 2.0, b = 3.0
